<a href="https://colab.research.google.com/github/Damilola-ops/PacMan-RL/blob/main/Pacman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get update > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg 
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install gym-retro
!pip install stable-baselines3[extra] optuna
!pip install opencv-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl xvfb
0 upgraded, 2 newly installed, 0 to remove and 76 not upgraded.
Need to get 1,281 kB of archives.
After this operation, 7,687 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.11 [785 kB]
Fetched 1,281 kB in 2s (720 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 155673 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpackin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162.0 MB 34 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 4.2 MB/s 
     |████████████████████████████████| 308 kB 29.7 MB/s 
     |████████████████████████████████| 209 kB 49.5 MB/s 
     |████████████████████████████████| 81 kB 9.3 MB/s 
     |████████████████████████████████| 78 kB 6.4 MB/s 
     |████████████████████████████████| 112 kB 42.5 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 147 kB 56.4 MB/s 
     |████████████████████████████████| 1.5 MB 61.1 MB/s 
     |████████████████████████████████| 1.1 MB 42.7 MB/s 
     |████████████████████████████████| 1.6 MB 37.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metad

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#navigating  to the directory where the our game ROM  is 
!cd '/content/drive/MyDrive/ROMS'

In [4]:
#importing our game rom
!python3 -m retro.import '/content/drive/MyDrive/ROMS'

Importing SpaceInvaders-Snes
Importing SpaceInvaders91-Genesis
Importing PacManNamco-Nes
Importing MsPacMan-Genesis
Importing StreetFighterIISpecialChampionEdition-Genesis
Importing MsPacMan-Genesis
Imported 6 games


In [13]:
#importing the necessary libraries 
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
from gym.wrappers import RecordVideo
gymlogger.set_level(40) #error only
import numpy as np
import random
import matplotlib
import time 
#for visualizing frames 
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
import retro
import optuna
import torch
# PPO algo for RL
from stable_baselines3 import PPO
# Bring in the eval policy method for metric calculation
from stable_baselines3.common.evaluation import evaluate_policy
# Import the sb3 monitor for logging 
# Import the vec wrappers to vectorize and frame stack
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from IPython import display as ipythondisplay

In [6]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [18]:
#a function to record and display the content of our environment as a video
def showvid():
  mp4list = glob.glob('video/*.mp4')
  if  len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4,'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else:
    print('could not find video')

def wrap_env(env):
  env=RecordVideo(env,'./video')
  return env





In [21]:
env = wrap_env(retro.make(game='MsPacMan-Genesis'))


In [20]:
env.close()

In [22]:
observation = env.reset()

while True:
  
    env.render()
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action) 
   
        
    if done: 
      break;
            
env.close()
showvid()

In [24]:
from gym import Env
from gym.spaces import MultiBinary,Box
import cv2 #for grayscaling



In [31]:
#creating a custom environment 
class PacMan(Env):
  def __init__(self):
    super().__init__()
    self.game_env = retro.make(game='MsPacMan-Genesis')
    self.action_space = MultiBinary(12)
    self.obs_space = Box(low=0,high=255,shape=(84,84,1),dtype = np.uint8)
    
  def reset(self):
    #return the first frame 
    obs = self.game.reset()
    obs = self.preprocess(obs)
    self.previous_frame = obs 
  def preprocess(self,observation):
    #Grayscaling our frames 
    gray = cv2.ctColor(observation,cv2.COLOR_BGR2GRAY)
    #resize 
    resize = cv2.resize(gray,(84,84),interpolation=cv2.INTER_CUBIC)
    #add channels value 
    channels = np.reshape(resize,(84,84,1))
    return channels 
  def step(self,action):
    #Take a step 
    obs,reward,done,info = self.game.step(action)
    obs = self.preprocess(obs)
    #frame delta 
    frame_delta = obs - self.previous_frame 
    self.previous_frame = obs 
    #reshape the reward function 
    reward = info['score'] - self.score
    self.score = info['score']

    return frame_delta,reward,done,info 
  def render(self,*args,**kwargs):
    self.game.render()
  def close(self):
    self.game.close()





In [32]:
#an instance of our custom environment 
env = PacMan()

In [33]:
env.obs_space.shape

(84, 84, 1)

In [34]:
log_dir = './logs/'
opt_dir = './opt/'

In [36]:
from stable_baselines3.common.monitor import Monitor

In [35]:
#function to return test hyperparameters - define the object function
def optimize_model(trial):
  return {
      'n_steps' : trial.suggest_int('n_steps','2048,8192'),
      'gamma' : trial.suggest_loguniform('gamma',0.8,0.9999),
      'learning_rate': trial.suggest_loguniform('learning_rate',1e-5,1e-4),
      'clip_range' : trial.suggest_uniform('clip_range',0.1,0.4),
      'gae_lambda': trial.suggest_uniform('gae_lambda',0.8,0.99)
  }

In [ ]:
save_path = os.path.join(opt_dr,'trial_{}_best_model'.format(1))

In [ ]:
#run a training loop and return mean reward 
def optimize_agent(trial):
  try:
    model_params = optimize_model(trial)
    #create environment 
    env = PacMan 
    env = Monitor(env,log_dir)
    env = DummyVecEnv([lambda:env])
    env = VecFrameStack(env,4,channels_order='last')
    
    #create RL algorithm
    model = PPO('CnnPolicy',env,tensorboard_log=log_dir,verbose=0,**model_params)
    model.learn(total_timesteps=30000)
    #evaluate model 
    mean_reward,_ = evaluate_policy(model,env,n_eval_episodes=5)
    env.close()
    save_path = os.path.join(opt_dir,'trial_{}_best_model'.format(trial.number))
    return mean_reward
  except Exception as e :
    return -1000 

In [ ]:
#creating the experiment 
study = optuna.create_study(direction='maximize')
study.optimize(optimize_agent,n_trials=10,n_jobs=1)


In [ ]:
study.best_params

In [ ]:
model = PPO.load(os.path.join(opt_dir,'trial_5_best_model.zip'))

In [ ]:
#CallBack 
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
checkpoint_dir = './train/'
callback = TrainAndLoggingCallback(check_freq=1000,save_path=checkpoint_dir)

# Training our Model 

In [ ]:
#create env 
env = PacMan()
env = Monitor(env,log_dir)
env = DummyVecEnv([lambda:env])
env = VecFrameStack(env,4,channels_order='last')


In [ ]:
model_params = study.best_params 
model_params['n_steps'] = 7488
model_params

In [ ]:
model = PPO('CnnPolicy',env,tensorboard_log=log_dir,verbose=1,**model_params)

In [ ]:
model.load(os.path.join(opt_dir,'trial_5_best_model.zip'))


In [ ]:
#start training
model.learn(total_timesteps=100000, callback=callback)

# Evaluate our Model

In [ ]:
model = PPO.load('./opt/trial_5_best_model.zip')
mean_reward,_ = evaluate_policy(model,env,render=True,n_eval_episodes=1)

In [ ]:
mean_reward

#Testing out our model 

In [ ]:
obs = env.reset()
obs.shape

In [ ]:
env.step(model.predict(obs)[0])

In [ ]:
obs = env.reset()
while True :
  env.render
  action = model.predict(obs)[0]
  obs,reward,done,info = env.step(action)
  time.sleep(0.01)
  print(reward)
  if done:
    env.reset()




In [ ]:
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)
        time.sleep(0.01)
        print(reward)